### Import Libraries

Use Jupyter Notebook as Spark IDE

In [1]:
import findspark
findspark.init()

Import required libraries

In [2]:
from pyspark import SparkContext
from pyspark.sql import SparkSession

In [3]:
sc=SparkContext.getOrCreate()
ss=SparkSession(sc)

### Load Data

Load data

In [4]:
data=ss.read.csv("../data/titanic.csv",inferSchema=True, header=True)

In [5]:
# data.cache() 

Get data structure

In [6]:
data.printSchema()

root
 |-- pclass: integer (nullable = true)
 |-- survived: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- age: double (nullable = true)
 |-- sibsp: integer (nullable = true)
 |-- parch: integer (nullable = true)
 |-- ticket: string (nullable = true)
 |-- fare: double (nullable = true)
 |-- cabin: string (nullable = true)
 |-- embarked: string (nullable = true)
 |-- boat: string (nullable = true)
 |-- body: integer (nullable = true)
 |-- home.dest: string (nullable = true)



Get first 5 records

In [7]:
data.show(5)

+------+--------+--------------------+------+------+-----+-----+------+--------+-------+--------+----+----+--------------------+
|pclass|survived|                name|   sex|   age|sibsp|parch|ticket|    fare|  cabin|embarked|boat|body|           home.dest|
+------+--------+--------------------+------+------+-----+-----+------+--------+-------+--------+----+----+--------------------+
|     1|       1|Allen, Miss. Elis...|female|  29.0|    0|    0| 24160|211.3375|     B5|       S|   2|null|        St Louis, MO|
|     1|       1|Allison, Master. ...|  male|0.9167|    1|    2|113781|  151.55|C22 C26|       S|  11|null|Montreal, PQ / Ch...|
|     1|       0|Allison, Miss. He...|female|   2.0|    1|    2|113781|  151.55|C22 C26|       S|null|null|Montreal, PQ / Ch...|
|     1|       0|Allison, Mr. Huds...|  male|  30.0|    1|    2|113781|  151.55|C22 C26|       S|null| 135|Montreal, PQ / Ch...|
|     1|       0|Allison, Mrs. Hud...|female|  25.0|    1|    2|113781|  151.55|C22 C26|       S|

In [8]:
data.columns

['pclass',
 'survived',
 'name',
 'sex',
 'age',
 'sibsp',
 'parch',
 'ticket',
 'fare',
 'cabin',
 'embarked',
 'boat',
 'body',
 'home.dest']

Describe the data

In [9]:
data.describe().show(truncate=True, vertical=True)

-RECORD 0-------------------------
 summary   | count                
 pclass    | 1309                 
 survived  | 1309                 
 name      | 1309                 
 sex       | 1309                 
 age       | 1046                 
 sibsp     | 1309                 
 parch     | 1309                 
 ticket    | 1309                 
 fare      | 1308                 
 cabin     | 295                  
 embarked  | 1307                 
 boat      | 486                  
 body      | 121                  
 home.dest | 745                  
-RECORD 1-------------------------
 summary   | mean                 
 pclass    | 2.294881588999236    
 survived  | 0.3819709702062643   
 name      | null                 
 sex       | null                 
 age       | 29.8811345124283     
 sibsp     | 0.4988540870893812   
 parch     | 0.3850267379679144   
 ticket    | 249039.1368861024    
 fare      | 33.29547928134572    
 cabin     | null                 
 embarked  | null   

### Preprocess Data

Rename columns

In [10]:
data=data.withColumnRenamed('home.dest','homedest')

Check for missing values

In [11]:
from pyspark.sql import functions
for i in data.columns:
    print(i,data.where(functions.col(i).isNull()).count())

pclass 1
survived 1
name 1
sex 1
age 264
sibsp 1
parch 1
ticket 1
fare 2
cabin 1015
embarked 3
boat 824
body 1189
homedest 565


Get records with missing data for sex column

In [12]:
data.where(data['age'].isNull()).show()

+------+--------+--------------------+------+----+-----+-----+--------+--------+-----+--------+----+----+--------------------+
|pclass|survived|                name|   sex| age|sibsp|parch|  ticket|    fare|cabin|embarked|boat|body|            homedest|
+------+--------+--------------------+------+----+-----+-----+--------+--------+-----+--------+----+----+--------------------+
|     1|       0| Baumann, Mr. John D|  male|null|    0|    0|PC 17318|  25.925| null|       S|null|null|        New York, NY|
|     1|       1|"Bradley, Mr. Geo...|  male|null|    0|    0|  111427|   26.55| null|       S|   9|null|     Los Angeles, CA|
|     1|       0|Brewe, Dr. Arthur...|  male|null|    0|    0|  112379|    39.6| null|       C|null|null|    Philadelphia, PA|
|     1|       0|Cairns, Mr. Alexa...|  male|null|    0|    0|  113798|    31.0| null|       S|null|null|                null|
|     1|       1|Cassebeer, Mrs. H...|female|null|    0|    0|   17770| 27.7208| null|       C|   5|null|      

Check Data Type

In [13]:
data.dtypes

[('pclass', 'int'),
 ('survived', 'int'),
 ('name', 'string'),
 ('sex', 'string'),
 ('age', 'double'),
 ('sibsp', 'int'),
 ('parch', 'int'),
 ('ticket', 'string'),
 ('fare', 'double'),
 ('cabin', 'string'),
 ('embarked', 'string'),
 ('boat', 'string'),
 ('body', 'int'),
 ('homedest', 'string')]

Get the mean age

In [22]:
from pyspark.sql.functions import mean
data.select(mean('age')).collect()[0][0]

23.85928755725191

Impute missing age value with mean age

In [23]:
data=data.na.fill(data.select(mean('age')).collect()[0][0],subset=['age'])

In [24]:
# Check missing age records
data.where(data['age'].isNull()).show()

+------+--------+----+---+---+-----+-----+------+----+-----+--------+----+----+--------+---------+--------------+
|pclass|survived|name|sex|age|sibsp|parch|ticket|fare|cabin|embarked|boat|body|homedest|sex_index|embarked_index|
+------+--------+----+---+---+-----+-----+------+----+-----+--------+----+----+--------+---------+--------------+
+------+--------+----+---+---+-----+-----+------+----+-----+--------+----+----+--------+---------+--------------+



Convert string to numeric type

In [19]:
data=data.na.fill('N',subset=['embarked'])
# data=data.withColumn('sex',data.sex.cast('int'))
data.dtypes

[('pclass', 'int'),
 ('survived', 'int'),
 ('name', 'string'),
 ('sex', 'string'),
 ('age', 'double'),
 ('sibsp', 'int'),
 ('parch', 'int'),
 ('ticket', 'string'),
 ('fare', 'double'),
 ('cabin', 'string'),
 ('embarked', 'string'),
 ('boat', 'string'),
 ('body', 'int'),
 ('homedest', 'string'),
 ('sex_index', 'double'),
 ('embarked_index', 'double')]

In [16]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer

indexers = [StringIndexer(inputCol=column, outputCol=column+"_index").fit(data) for column in ["sex","embarked"]]
pipeline = Pipeline(stages=indexers)
data = pipeline.fit(data).transform(data)


In [17]:
data.show(vertical=True)

-RECORD 0------------------------------
 pclass         | 1                    
 survived       | 1                    
 name           | Allen, Miss. Elis... 
 sex            | female               
 age            | 29.0                 
 sibsp          | 0                    
 parch          | 0                    
 ticket         | 24160                
 fare           | 211.3375             
 cabin          | B5                   
 embarked       | S                    
 boat           | 2                    
 body           | null                 
 homedest       | St Louis, MO         
 sex_index      | 1.0                  
 embarked_index | 0.0                  
-RECORD 1------------------------------
 pclass         | 1                    
 survived       | 1                    
 name           | Allison, Master. ... 
 sex            | male                 
 age            | 0.9167               
 sibsp          | 1                    
 parch          | 2                    


Create Feature

In [18]:
feature=['pclass','sibsp','parch']